In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import torch
import torch.optim as optim
import torch.nn as nn
import datetime
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import torch.nn.functional as F
# from data_generation.image_classication import generate_dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [2]:
class StepByStep(object):
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        self.losses = []
        self.val_loss = []
        self.epochs = 0
        self.train_step_fn = self.train_step_fn()
        self.val_step_fn = self.val_step_fn()
    def to(self, device):
        try:
            self.device = device

            self.model.to(self.device)

        except RuntimeError:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            print("can't send it to ${device}, sending to ${self.device} instead")
            self.model.to(self.device)
            self.train_loader = None
            self.valid_loader = None
            self.writer = None
    def set_loaders(self, train_loader, val_loader = None):
        self.train_loader = train_loader
        self.valid_loader = val_loader
    def set_tensorboard(self, name, folder='runs'):
        suffix = datetime.datetime.now().strftime('%Y%d%H%M%S')
        self.writer = SummaryWriter(f'{folder}/{name}_{suffix}')

In [3]:
#NCHW:: Number of sample : Channel : Height : Width
single = np.array([[[[5, 0, 8, 7, 8, 1],
                     [1, 9, 5, 0, 7, 7],
                     [6, 0, 2, 4, 6, 6],
                     [9, 7, 6, 6, 8, 4],
                     [8, 3, 8, 5, 1, 3],
                     [7, 2, 7, 0, 1, 0]]]])
single.shape

(1, 1, 6, 6)

In [4]:
identity = np.array([[[[0, 0, 0],
                       [0, 1, 0],
                       [0, 0, 0]]]])
identity.shape

(1, 1, 3, 3)

In [5]:
region = single[:, :, 0:3, 0:3]
print('region::', region)
filter_region = region * identity
print('after element-wise multiplication:: ', filter_region)
total = filter_region.sum()
total
print('total::', total)

region:: [[[[5 0 8]
   [1 9 5]
   [6 0 2]]]]
after element-wise multiplication::  [[[[0 0 0]
   [0 9 0]
   [0 0 0]]]]
total:: 9


In [6]:
image = torch.as_tensor(single).float()
kernel_identity = torch.as_tensor(identity).float()

In [7]:
convolved = F.conv2d(image, kernel_identity, stride=1)
convolved

tensor([[[[9., 5., 0., 7.],
          [0., 2., 4., 6.],
          [7., 6., 6., 8.],
          [3., 8., 5., 1.]]]])

In [8]:
convo = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1)(image)
convo

tensor([[[[-3.0320, -5.0933, -1.0802, -1.1762],
          [-3.9790,  0.2137, -3.6081, -5.5315],
          [-2.6562, -5.2238, -3.5981, -3.2906],
          [-0.8202, -3.2799, -2.8878,  0.0581]]]],
       grad_fn=<ConvolutionBackward0>)

In [11]:
conv_multiple = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1)(image)
conv_multiple

tensor([[[[ -5.3211,  -7.8240,  -9.7881,  -7.4611],
          [ -6.6923,  -6.0765,  -7.4523,  -6.9460],
          [ -8.7099,  -7.0304,  -6.1616,  -7.4394],
          [-10.6734,  -7.7725,  -7.5443,  -6.0781]],

         [[  2.8981,  -3.0997,  -1.8501,   1.8219],
          [ -1.4432,   0.9954,   2.7486,   1.5626],
          [  2.5167,   2.2655,   2.2716,   0.0851],
          [ -0.7554,   0.8644,  -2.0244,  -3.2278]]]],
       grad_fn=<ConvolutionBackward0>)